<a href="https://colab.research.google.com/github/MooiWhite/AyPT/blob/main/Proyecto_Final_FALSAS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!python3 -m spacy download en_core_web_md

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 96.4 MB 1.5 MB/s 
  Created wheel for en-core-web-md: filename=en_core_web_md-2.2.5-py3-none-any.whl size=98051301 sha256=523825ed97ada5c5d1b017ca61a39f10461ac1ef423fdfba6095802ea781bedb
  Stored in directory: /tmp/pip-ephem-wheel-cache-6f_doojw/wheels/69/c5/b8/4f1c029d89238734311b3269762ab2ee325a42da2ce8edb997
Successfully built en-core-web-md
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_md')


In [2]:
!pip install pyldavis

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.7 MB 5.3 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
    Preparing wheel metadata ... done
  Created wheel for pyldavis: filename=pyLDAvis-3.3.1-py2.py3-none-any.whl size=136898 sha256=5642ef0cac0198b7fb1038f2fe2e248fe4e51db504acbf8d5e166e936864dbda
  Stored in directory: /root/.cache/pip/wheels/c9/21/f6/17bcf2667e8a68532ba2fbf6d5c72fdf4c7f7d9abfa4852d2f
Successfully built pyldavis


In [3]:
import pandas as pd
import glob
import os
import spacy
import gensim
import gensim.corpora as corpora
import pyLDAvis
import pyLDAvis.gensim_models
import en_core_web_sm

/usr/local/lib/python3.7/dist-packages/past/types/oldstr.py:5: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  from collections import Iterable


In [4]:
pyLDAvis.enable_notebook()

In [5]:
nlp = en_core_web_sm.load()

/usr/local/lib/python3.7/dist-packages/catalogue.py:138: DeprecationWarning: SelectableGroups dict interface is deprecated. Use select.
  for entry_point in AVAILABLE_ENTRY_POINTS.get(self.entry_point_namespace, []):
/usr/local/lib/python3.7/dist-packages/catalogue.py:138: DeprecationWarning: SelectableGroups dict interface is deprecated. Use select.
  for entry_point in AVAILABLE_ENTRY_POINTS.get(self.entry_point_namespace, []):
/usr/local/lib/python3.7/dist-packages/catalogue.py:126: DeprecationWarning: SelectableGroups dict interface is deprecated. Use select.
  for entry_point in AVAILABLE_ENTRY_POINTS.get(self.entry_point_namespace, []):
/usr/local/lib/python3.7/dist-packages/catalogue.py:138: DeprecationWarning: SelectableGroups dict interface is deprecated. Use select.
  for entry_point in AVAILABLE_ENTRY_POINTS.get(self.entry_point_namespace, []):


In [6]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [8]:
df = pd.concat(map(pd.read_csv, glob.glob(os.path.join("/content/drive/MyDrive/Semestre 2022-2/AyPT/fake_or_real_news.csv"))), ignore_index= True)

In [9]:
print(df)

      Unnamed: 0                                              title  \
0           8476                       You Can Smell Hillary’s Fear   
1          10294  Watch The Exact Moment Paul Ryan Committed Pol...   
2           3608        Kerry to go to Paris in gesture of sympathy   
3          10142  Bernie supporters on Twitter erupt in anger ag...   
4            875   The Battle of New York: Why This Primary Matters   
...          ...                                                ...   
6330        4490  State Department says it can't find emails fro...   
6331        8062  The ‘P’ in PBS Should Stand for ‘Plutocratic’ ...   
6332        8622  Anti-Trump Protesters Are Tools of the Oligarc...   
6333        4021  In Ethiopia, Obama seeks progress on peace, se...   
6334        4330  Jeb Bush Is Suddenly Attacking Trump. Here's W...   

                                                   text label  
0     Daniel Greenfield, a Shillman Journalism Fello...  FAKE  
1     Google Pinter

In [16]:
#quitando las filas que sean noticias falsas
df_s1 = df.drop(df.query("label=='REAL'").index)

In [17]:
print(df_s1)

      Unnamed: 0                                              title  \
0           8476                       You Can Smell Hillary’s Fear   
1          10294  Watch The Exact Moment Paul Ryan Committed Pol...   
3          10142  Bernie supporters on Twitter erupt in anger ag...   
5           6903                                        Tehran, USA   
6           7341  Girl Horrified At What She Watches Boyfriend D...   
...          ...                                                ...   
6326        6143  DOJ COMPLAINT: Comey Under Fire Over Partisan ...   
6328        9337  Radio Derb Is On The Air–Leonardo And Brazil’s...   
6329        8737  Assange claims ‘crazed’ Clinton campaign tried...   
6331        8062  The ‘P’ in PBS Should Stand for ‘Plutocratic’ ...   
6332        8622  Anti-Trump Protesters Are Tools of the Oligarc...   

                                                   text label  
0     Daniel Greenfield, a Shillman Journalism Fello...  FAKE  
1     Google Pinter

In [18]:
#ds_s1 son los datos sin las filas con la etiqueta FAKE
news = df_s1.groupby(["label", "title"], sort=False).apply(lambda x: "\n".join(x["text"]))

In [19]:
news.head()

label  title                                                                                
FAKE   You Can Smell Hillary’s Fear                                                             Daniel Greenfield, a Shillman Journalism Fello...
       Watch The Exact Moment Paul Ryan Committed Political Suicide At A Trump Rally (VIDEO)    Google Pinterest Digg Linkedin Reddit Stumbleu...
       Bernie supporters on Twitter erupt in anger against the DNC: 'We tried to warn you!'     — Kaydee King (@KaydeeKing) November 9, 2016 T...
       Tehran, USA                                                                                \nI’m not an immigrant, but my grandparents ...
       Girl Horrified At What She Watches Boyfriend Do After He Left FaceTime On                Share This Baylee Luciani (left), Screenshot o...
dtype: object

In [20]:
new_df = pd.DataFrame()
new_df["text"] = news

In [21]:
new_df["LEMMATISED"] = news.apply(lambda x: " ".join(w.lemma_ for w in nlp(x) if w.pos_ in ["ADJ"]))

Se han truncado las últimas 5000 líneas del flujo de salida.
/usr/local/lib/python3.7/dist-packages/catalogue.py:138: DeprecationWarning: SelectableGroups dict interface is deprecated. Use select.
  for entry_point in AVAILABLE_ENTRY_POINTS.get(self.entry_point_namespace, []):
/usr/local/lib/python3.7/dist-packages/catalogue.py:138: DeprecationWarning: SelectableGroups dict interface is deprecated. Use select.
  for entry_point in AVAILABLE_ENTRY_POINTS.get(self.entry_point_namespace, []):
/usr/local/lib/python3.7/dist-packages/catalogue.py:138: DeprecationWarning: SelectableGroups dict interface is deprecated. Use select.
  for entry_point in AVAILABLE_ENTRY_POINTS.get(self.entry_point_namespace, []):
/usr/local/lib/python3.7/dist-packages/catalogue.py:138: DeprecationWarning: SelectableGroups dict interface is deprecated. Use select.
  for entry_point in AVAILABLE_ENTRY_POINTS.get(self.entry_point_namespace, []):
/usr/local/lib/python3.7/dist-packages/catalogue.py:138: DeprecationWar

In [22]:
new_df

text  \
label title                                                                                                   
FAKE  You Can Smell Hillary’s Fear                        Daniel Greenfield, a Shillman Journalism Fello...   
      Watch The Exact Moment Paul Ryan Committed Poli...  Google Pinterest Digg Linkedin Reddit Stumbleu...   
      Bernie supporters on Twitter erupt in anger aga...  — Kaydee King (@KaydeeKing) November 9, 2016 T...   
      Tehran, USA                                           \nI’m not an immigrant, but my grandparents ...   
      Girl Horrified At What She Watches Boyfriend Do...  Share This Baylee Luciani (left), Screenshot o...   
...                                                                                                     ...   
      Will the Media Reset After the Election or Are ...  Written by Peter Van Buren   venerable New Yor...   
      DOJ COMPLAINT: Comey Under Fire Over Partisan W...  DOJ COMPLAINT: Comey Under Fire Over Partisan ...   
      Radio Derb Is On The Air–Leonardo And Brazil’s ...                                                      
      The ‘P’ in PBS Should Stand for ‘Plutocratic’ o...  The ‘P’ in PBS Should Stand for ‘Plutocratic’ ...   
      Anti-Trump Protesters Are Tools of the Oligarch...   Anti-Trump Protesters Are Tools of the Oligar...   

                                                                                                 LEMMATISED  
label title                                                                                                  
FAKE  You Can Smell Hillary’s Fear                        radical final unprecedented unprecedented majo...  
      Watch The Exact Moment Paul Ryan Committed Poli...  fundamental staggering political - - reversed ...  
      Bernie supporters on Twitter erupt in anger aga...  dem same tight outspoken contentious democrati...  
      Tehran, USA                                         More central american little ancestral old abj...  
      Girl Horrified At What She Watches Boyfriend Do...  right close regular endearing regular other av...  
...                                                                                                     ...  
      Will the Media Reset After the Election or Are ...  venerable apparent overall many actual bio lon...  
      DOJ COMPLAINT: Comey Under Fire Over Partisan W...  doj entire democratic presidential same privat...  
      Radio Derb Is On The Air–Leonardo And Brazil’s ...                                                     
      The ‘P’ in PBS Should Stand for ‘Plutocratic’ o...  absurd regular free corporate corporate financ...  
      Anti-Trump Protesters Are Tools of the Oligarch...  anti - trump old old anti - trump presidential...  

[3107 rows x 2 columns]

In [23]:
new_df["PREPROCESSED"] = new_df["LEMMATISED"].apply(lambda x: gensim.utils.simple_preprocess(x, deacc=True, min_len=3))

In [24]:
new_df.tail()

text  \
label title                                                                                                   
FAKE  Will the Media Reset After the Election or Are ...  Written by Peter Van Buren   venerable New Yor...   
      DOJ COMPLAINT: Comey Under Fire Over Partisan W...  DOJ COMPLAINT: Comey Under Fire Over Partisan ...   
      Radio Derb Is On The Air–Leonardo And Brazil’s ...                                                      
      The ‘P’ in PBS Should Stand for ‘Plutocratic’ o...  The ‘P’ in PBS Should Stand for ‘Plutocratic’ ...   
      Anti-Trump Protesters Are Tools of the Oligarch...   Anti-Trump Protesters Are Tools of the Oligar...   

                                                                                                 LEMMATISED  \
label title                                                                                                   
FAKE  Will the Media Reset After the Election or Are ...  venerable apparent overall many actual bio lon...   
      DOJ COMPLAINT: Comey Under Fire Over Partisan W...  doj entire democratic presidential same privat...   
      Radio Derb Is On The Air–Leonardo And Brazil’s ...                                                      
      The ‘P’ in PBS Should Stand for ‘Plutocratic’ o...  absurd regular free corporate corporate financ...   
      Anti-Trump Protesters Are Tools of the Oligarch...  anti - trump old old anti - trump presidential...   

                                                                                               PREPROCESSED  
label title                                                                                                  
FAKE  Will the Media Reset After the Election or Are ...  [venerable, apparent, overall, many, actual, b...  
      DOJ COMPLAINT: Comey Under Fire Over Partisan W...  [doj, entire, democratic, presidential, same, ...  
      Radio Derb Is On The Air–Leonardo And Brazil’s ...                                                 []  
      The ‘P’ in PBS Should Stand for ‘Plutocratic’ o...  [absurd, regular, free, corporate, corporate, ...  
      Anti-Trump Protesters Are Tools of the Oligarch...  [anti, trump, old, old, anti, trump, president...

In [25]:
id2word = corpora.Dictionary(new_df["PREPROCESSED"])

In [26]:
len(id2word)

7162

In [27]:
new_df["CORPUS"] = new_df["PREPROCESSED"].apply(lambda x: id2word.doc2bow(x))

In [28]:
new_df.tail()

text  \
label title                                                                                                   
FAKE  Will the Media Reset After the Election or Are ...  Written by Peter Van Buren   venerable New Yor...   
      DOJ COMPLAINT: Comey Under Fire Over Partisan W...  DOJ COMPLAINT: Comey Under Fire Over Partisan ...   
      Radio Derb Is On The Air–Leonardo And Brazil’s ...                                                      
      The ‘P’ in PBS Should Stand for ‘Plutocratic’ o...  The ‘P’ in PBS Should Stand for ‘Plutocratic’ ...   
      Anti-Trump Protesters Are Tools of the Oligarch...   Anti-Trump Protesters Are Tools of the Oligar...   

                                                                                                 LEMMATISED  \
label title                                                                                                   
FAKE  Will the Media Reset After the Election or Are ...  venerable apparent overall many actual bio lon...   
      DOJ COMPLAINT: Comey Under Fire Over Partisan W...  doj entire democratic presidential same privat...   
      Radio Derb Is On The Air–Leonardo And Brazil’s ...                                                      
      The ‘P’ in PBS Should Stand for ‘Plutocratic’ o...  absurd regular free corporate corporate financ...   
      Anti-Trump Protesters Are Tools of the Oligarch...  anti - trump old old anti - trump presidential...   

                                                                                               PREPROCESSED  \
label title                                                                                                   
FAKE  Will the Media Reset After the Election or Are ...  [venerable, apparent, overall, many, actual, b...   
      DOJ COMPLAINT: Comey Under Fire Over Partisan W...  [doj, entire, democratic, presidential, same, ...   
      Radio Derb Is On The Air–Leonardo And Brazil’s ...                                                 []   
      The ‘P’ in PBS Should Stand for ‘Plutocratic’ o...  [absurd, regular, free, corporate, corporate, ...   
      Anti-Trump Protesters Are Tools of the Oligarch...  [anti, trump, old, old, anti, trump, president...   

                                                                                                     CORPUS  
label title                                                                                                  
FAKE  Will the Media Reset After the Election or Are ...  [(14, 1), (36, 1), (47, 2), (54, 2), (62, 3), ...  
      DOJ COMPLAINT: Comey Under Fire Over Partisan W...  [(19, 1), (20, 2), (21, 3), (31, 1), (44, 1), ...  
      Radio Derb Is On The Air–Leonardo And Brazil’s ...                                                 []  
      The ‘P’ in PBS Should Stand for ‘Plutocratic’ o...  [(6, 4), (7, 1), (14, 2), (24, 3), (26, 1), (3...  
      Anti-Trump Protesters Are Tools of the Oligarch...  [(6, 1), (24, 2), (31, 1), (34, 1), (41, 2), (...

In [30]:
new_df["CORPUS"].loc["FAKE", "DOJ COMPLAINT: Comey Under Fire Over Partisan Witch Hunt For Hillary (TWEETS/VIDEO)"]

[(19, 1),
 (20, 2),
 (21, 3),
 (31, 1),
 (44, 1),
 (50, 3),
 (54, 2),
 (57, 1),
 (62, 1),
 (85, 1),
 (104, 1),
 (111, 1),
 (118, 1),
 (194, 1),
 (206, 1),
 (282, 1),
 (378, 1),
 (402, 1),
 (429, 2),
 (432, 1),
 (549, 1),
 (621, 1),
 (682, 1),
 (950, 1),
 (966, 1),
 (1190, 1),
 (1564, 1),
 (2513, 1)]

In [31]:
len(new_df["CORPUS"].loc["FAKE", "DOJ COMPLAINT: Comey Under Fire Over Partisan Witch Hunt For Hillary (TWEETS/VIDEO)"])

28

In [32]:
id2word[749]

'lukewarm'

In [49]:
lda_model = gensim.models.ldamodel.LdaModel(
    corpus=new_df["CORPUS"],
    id2word=id2word,
    num_topics = 10,
    random_state=42,
    update_every=1,
    chunksize=100,
    passes=20,
    alpha="auto"
)

Se han truncado las últimas 5000 líneas del flujo de salida.
/usr/local/lib/python3.7/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/usr/local/lib/python3.7/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/usr/local/lib/python3.7/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += 

In [48]:
pyLDAvis.gensim_models.prepare(lda_model, new_df["CORPUS"], id2word)

/usr/local/lib/python3.7/dist-packages/pyLDAvis/_prepare.py:247: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  by='saliency', ascending=False).head(R).drop('saliency', 1)


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
4      0.216556  0.045335       1        1  40.764581
3      0.183483  0.087481       2        1  26.626462
0      0.088961 -0.222914       3        1  20.179801
1     -0.280954 -0.174280       4        1   7.595043
2     -0.208047  0.264378       5        1   4.834113, topic_info=          Term         Freq        Total Category  logprob  loglift
290    russian  1297.000000  1297.000000  Default  30.0000  30.0000
50   political  1334.000000  1334.000000  Default  29.0000  29.0000
417     former  1239.000000  1239.000000  Default  28.0000  28.0000
151   military   785.000000   785.000000  Default  27.0000  27.0000
41         new  2180.000000  2180.000000  Default  26.0000  26.0000
..         ...          ...          ...      ...      ...      ...
400      heavy    46.504731   122.451944   Topic5  -5.1535   2.0613
403     strong    62.936987   491.011915   Topic5  -4.8509   0.9751
662        low    56.797717   417.594234   Topic5  -4.9536   1.0345
604  effective    44.421168   137.961163   Topic5  -5.1993   1.8962
143       high    47.140578  1143.180813   Topic5  -5.1399  -0.1590

[286 rows x 6 columns], token_table=      Topic      Freq       Term
term                            
0         1  0.995079      about
1205      3  0.991213     absurd
790       2  0.997377     active
1805      5  0.976476    adverse
1054      4  0.993366     aerial
...     ...       ...        ...
78        2  0.658615      whole
7156      3  0.996179   withdrew
1282      1  0.990039  worldwide
79        2  0.999113      wrong
189       2  0.999442      young

[364 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[5, 4, 1, 2, 3])

In [35]:
new_df = new_df[["text", "LEMMATISED", "CORPUS"]]

In [36]:
new_df["TOPICS"] = new_df["CORPUS"].apply(lambda x: lda_model[x])

In [37]:
new_df.tail()

text  \
label title                                                                                                   
FAKE  Will the Media Reset After the Election or Are ...  Written by Peter Van Buren   venerable New Yor...   
      DOJ COMPLAINT: Comey Under Fire Over Partisan W...  DOJ COMPLAINT: Comey Under Fire Over Partisan ...   
      Radio Derb Is On The Air–Leonardo And Brazil’s ...                                                      
      The ‘P’ in PBS Should Stand for ‘Plutocratic’ o...  The ‘P’ in PBS Should Stand for ‘Plutocratic’ ...   
      Anti-Trump Protesters Are Tools of the Oligarch...   Anti-Trump Protesters Are Tools of the Oligar...   

                                                                                                 LEMMATISED  \
label title                                                                                                   
FAKE  Will the Media Reset After the Election or Are ...  venerable apparent overall many actual bio lon...   
      DOJ COMPLAINT: Comey Under Fire Over Partisan W...  doj entire democratic presidential same privat...   
      Radio Derb Is On The Air–Leonardo And Brazil’s ...                                                      
      The ‘P’ in PBS Should Stand for ‘Plutocratic’ o...  absurd regular free corporate corporate financ...   
      Anti-Trump Protesters Are Tools of the Oligarch...  anti - trump old old anti - trump presidential...   

                                                                                                     CORPUS  \
label title                                                                                                   
FAKE  Will the Media Reset After the Election or Are ...  [(14, 1), (36, 1), (47, 2), (54, 2), (62, 3), ...   
      DOJ COMPLAINT: Comey Under Fire Over Partisan W...  [(19, 1), (20, 2), (21, 3), (31, 1), (44, 1), ...   
      Radio Derb Is On The Air–Leonardo And Brazil’s ...                                                 []   
      The ‘P’ in PBS Should Stand for ‘Plutocratic’ o...  [(6, 4), (7, 1), (14, 2), (24, 3), (26, 1), (3...   
      Anti-Trump Protesters Are Tools of the Oligarch...  [(6, 1), (24, 2), (31, 1), (34, 1), (41, 2), (...   

                                                                                                     TOPICS  
label title                                                                                                  
FAKE  Will the Media Reset After the Election or Are ...  [(3, 0.13233769), (4, 0.45236266), (5, 0.09242...  
      DOJ COMPLAINT: Comey Under Fire Over Partisan W...  [(1, 0.025961775), (3, 0.07402675), (4, 0.3642...  
      Radio Derb Is On The Air–Leonardo And Brazil’s ...  [(0, 0.030947285), (1, 0.0193186), (2, 0.02339...  
      The ‘P’ in PBS Should Stand for ‘Plutocratic’ o...  [(0, 0.036699384), (1, 0.058776747), (2, 0.011...  
      Anti-Trump Protesters Are Tools of the Oligarch...  [(0, 0.07577214), (1, 0.08680831), (2, 0.03820...

In [38]:
new_df["MAIN_TOPIC"] = new_df.TOPICS.apply(lambda x: max(x, key=lambda y: y[1])[0])
new_df["MAIN_PERC"] = new_df.TOPICS.apply(lambda x: max(x, key=lambda y: y[1])[1])

In [39]:
new_df["MAIN_TOPIC"].value_counts()

4     2795
12      95
3       74
5       69
0       19
2       14
6       12
7       12
11       9
10       3
1        2
9        2
8        1
Name: MAIN_TOPIC, dtype: int64

In [40]:
new_df[new_df["MAIN_TOPIC"] == 11]

text  \
label title                                                                                                   
FAKE  Only Demented Imbeciles Want A Killary Reign Of...  in: Government Corruption , Special Interests ...   
      The Anatomy of Crisis and the Decline of US Empire  The Anatomy of Crisis and the Decline of US Em...   
      Carmel Institute celebrates 5th anniversary wit...  Carmel Institute celebrates 5th anniversary wi...   
      Climate Engineering And Cryosphere Collapse         geoengineeringwatch.org \nGlobal climate engin...   
      Economic Nationalism: Alternative To Globalism      in: Corporate Takeover , Economy & Business , ...   
      'Russia has no intention of attacking anyone -t...  October 28, 2016 112 While the Western press c...   
      Why Sprott Believes Gold Bullion Is A Mandatory...  Why Sprott Believes Gold Bullion Is A Mandator...   
      The Danger of Germany’s Current Account Surpluses   by Yves Smith \nYves here. Germany seems deter...   
      Hillary Clinton’s “Sudden Move” Of $1.8 Billion...  Email \n\nAn intriguing Ministry of Finance (M...   

                                                                                                 LEMMATISED  \
label title                                                                                                   
FAKE  Only Demented Imbeciles Want A Killary Reign Of...  dead hungry willing global inevitable economic...   
      The Anatomy of Crisis and the Decline of US Empire  multiple late racist unable economic vast blac...   
      Carmel Institute celebrates 5th anniversary wit...  5th rbth u.s.-russia 5th 5th historic metropol...   
      Climate Engineering And Cryosphere Collapse         global single great human global oblivious ong...   
      Economic Nationalism: Alternative To Globalism      corporate financial national economic own unch...   
      'Russia has no intention of attacking anyone -t...  western russian american presidential unlikely...   
      Why Sprott Believes Gold Bullion Is A Mandatory...  central complicated simple economic mandatory ...   
      The Danger of Germany’s Current Account Surpluses   determined contradictory large willing high fi...   
      Hillary Clinton’s “Sudden Move” Of $1.8 Billion...  intriguing elite western few old international...   

                                                                                                     CORPUS  \
label title                                                                                                   
FAKE  Only Demented Imbeciles Want A Killary Reign Of...  [(2, 1), (7, 1), (14, 2), (18, 1), (20, 1), (2...   
      The Anatomy of Crisis and the Decline of US Empire  [(7, 1), (20, 2), (30, 1), (31, 2), (34, 1), (...   
      Carmel Institute celebrates 5th anniversary wit...  [(24, 1), (36, 1), (51, 1), (85, 3), (189, 1),...   
      Climate Engineering And Cryosphere Collapse         [(6, 1), (7, 1), (20, 1), (30, 1), (34, 2), (3...   
      Economic Nationalism: Alternative To Globalism      [(20, 1), (24, 7), (30, 1), (36, 1), (38, 1), ...   
      'Russia has no intention of attacking anyone -t...  [(20, 2), (41, 5), (45, 1), (47, 2), (48, 4), ...   
      Why Sprott Believes Gold Bullion Is A Mandatory...  [(0, 1), (5, 1), (20, 1), (23, 1), (36, 3), (4...   
      The Danger of Germany’s Current Account Surpluses   [(6, 3), (7, 1), (21, 2), (24, 1), (31, 1), (3...   
      Hillary Clinton’s “Sudden Move” Of $1.8 Billion...  [(14, 2), (44, 1), (54, 1), (83, 1), (117, 1),...   

                                                                                                     TOPICS  \
label title                                                                                                   
FAKE  Only Demented Imbeciles Want A Killary Reign Of...  [(0, 0.010846965), (3, 0.1928452), (4, 0.23593...   
      The Anatomy of Crisis and the Decline of US Empire  [(0, 0.08519474), (1, 0.015557285), (3, 0.0924

In [42]:
new_df.loc["FAKE", "'Russia has no intention of attacking anyone -this is absurd,' says Vladimir Putin"]["TOPICS"]

[(1, 0.016102353),
 (3, 0.19351184),
 (4, 0.20662376),
 (5, 0.18155044),
 (6, 0.022069335),
 (7, 0.010383577),
 (10, 0.035487484),
 (11, 0.22742176),
 (12, 0.090035595)]

In [43]:
lda_model.show_topic(2) + lda_model.show_topic(10) + lda_model.show_topic(0)

[('illegal', 0.12131388),
 ('pointless', 0.06290629),
 ('retired', 0.02636394),
 ('associate', 0.025206685),
 ('preferable', 0.025168464),
 ('proper', 0.022525845),
 ('sentimental', 0.02064155),
 ('peaceful', 0.019223016),
 ('strong', 0.017423684),
 ('bold', 0.017138608),
 ('human', 0.0911042),
 ('critical', 0.04310484),
 ('physical', 0.025370294),
 ('central', 0.025059873),
 ('natural', 0.021927945),
 ('indian', 0.01847957),
 ('daily', 0.01596582),
 ('mental', 0.015810076),
 ('psychological', 0.01259666),
 ('replete', 0.012345883),
 ('black', 0.083580375),
 ('white', 0.067309186),
 ('muslim', 0.04475047),
 ('anti', 0.04436319),
 ('neoconservative', 0.039057184),
 ('jewish', 0.031871706),
 ('african', 0.028596265),
 ('horrific', 0.023962205),
 ('racist', 0.023942817),
 ('french', 0.021467667)]

In [44]:
new_df[new_df["MAIN_PERC"] > 0.75]

text  \
label title                                                                                                   
FAKE  Ying and Yang (the Gold and Silver Set-Up)          Ying and Yang (the Gold and Silver Set-Up) Pos...   
      CNN: One voter can make a difference by voting ...  Channel list \nFollowing hurricane Matthew's f...   
      Tweetwave                                           This time it's true no pantsTweetwave More Of ...   
      Is This Why The Smash In Gold & Silver Is Happe...  174 Views November 11, 2016 GOLD , KWN , KWN I...   
      How To Plan Farmer’s Calendar All Year Round          14, 2016 How To Plan Farmer’s Calendar All Y...   
      Hillary Is DONE After Blunt Cowboy Makes Shocki...  Hillary Is DONE After Blunt Cowboy Makes Shock...   
      Dead Voter Drive: video & petition to stop dece...  Channel list \nFollowing hurricane Matthew's f...   
      It's over! Trump is out - see today's final kil...  Channel list \nFollowing hurricane Matthew's f...   
      Trump Rally On Hollywood Boulevard 10/29/16         Trump Rally On Hollywood Boulevard 10/29/16 Us...   
      The Glorious New PC NFL Starts Next Monday!         Afraid of "dangerous" Trump presidency, protes...   

                                                                                                 LEMMATISED  \
label title                                                                                                   
FAKE  Ying and Yang (the Gold and Silver Set-Up)          new chinese much few positive economic likely ...   
      CNN: One voter can make a difference by voting ...  devastate many non - undecided racist sexist p...   
      Tweetwave                                           true More registered previous uncontested cons...   
      Is This Why The Smash In Gold & Silver Is Happe...  shocking long huge correct many many chinese n...   
      How To Plan Farmer’s Calendar All Year Round        true busy next such other impromptu great succ...   
      Hillary Is DONE After Blunt Cowboy Makes Shocki...  shocking live live shocking safe democratic cr...   
      Dead Voter Drive: video & petition to stop dece...  devastate many non - undecided racist sexist p...   
      It's over! Trump is out - see today's final kil...  devastate many non - undecided racist sexist p...   
      Trump Rally On Hollywood Boulevard 10/29/16         previous uncontested constant constant other o...   
      The Glorious New PC NFL Starts Next Monday!         afraid dangerous trump foreign unexpected spon...   

                                                                                                     CORPUS  \
label title                                                                                                   
FAKE  Ying and Yang (the Gold and Silver Set-Up)          [(6, 1), (30, 5), (31, 2), (34, 1), (36, 3), (...   
      CNN: One voter can make a difference by voting ...  [(6, 9), (7, 1), (18, 1), (20, 1), (21, 4), (2...   
      Tweetwave                                           [(6, 9), (7, 1), (18, 1), (20, 1), (21, 4), (2...   
      Is This Why The Smash In Gold & Silver Is Happe...  [(22, 1), (24, 2), (28, 2), (30, 2), (31, 2), ...   
      How To Plan Farmer’s Calendar All Year Round        [(6, 1), (20, 1), (24, 3), (30, 1), (31, 2), (...   
      Hillary Is DONE After Blunt Cowboy Makes Shocki...  [(18, 1), (25, 1), (30, 1), (41, 1), (45, 1), ...   
      Dead Voter Drive: video & petition to stop dece...  [(6, 9), (7, 1), (18, 1), (20, 1), (21, 4), (2...   
      It's over! Trump is out - see today's final kil...  [(6, 9), (7, 1), (18, 1), (20, 1), (21, 4), (2...   
      Trump Rally On Hollywood Boulevard 10/29/16         [(6, 9), (7, 1), (18, 1), (20, 1), (21, 4), (2...   
      The Glorious New PC NFL Starts Next Monday!         [(1, 1), (6, 9), (7, 1), (18, 1), (20, 1), (21...   

                                                                                                     TOP